# 환경세팅 - API 연결

In [1]:
# 📌
!pip install --upgrade openai
!pip install langchain
!pip install langchain_openai


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip install --upgrade pip


In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate
import getpass
import os
from langchain_openai import ChatOpenAI
import pandas as pd
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

import pandas as pd
from rouge_score import rouge_scorer
import re
import requests
import openai
from datetime import datetime

In [2]:
# 환경변수에 OpenAI API 키 저장 (사용자 입력으로 안전하게)
os.environ["OPENAI_API_KEY"] = getpass.getpass()
llm = ChatOpenAI(model="gpt-4o")

········


* 연결 완료

# Input data 불러오기 

## 뉴스 데이터 

In [4]:
news_ni = pd.read_csv('data/news/니켈본문_크롤링.csv')
news_li = pd.read_csv('data/news/리튬본문_크롤링.csv')
news_co = pd.read_csv('data/news/코발트본문_크롤링.csv')
news_ma= pd.read_csv('data/news/망간본문_크롤링.csv')

In [5]:
news_ni.head(2)

,date,title,link,content
0,2024-02-15 15:30:01,김준형 포스코퓨처엠 사장 “광양 하이니켈 NCA 공장 이달 착공…흑연 국산화 고민”,https://n.news.naver.com/mnews/article/016/000...,[\n15일 배터리협회 총회서 현장 인터뷰 이달 착공식·2025년 양산 예정…SDI...
1,2024-02-22 16:34:01,"포스코퓨처엠, 광양 고부가 하이니켈 NCA 양극재 공장 착공",https://n.news.naver.com/mnews/article/079/000...,[\n\n\n\n\n포스코퓨처엠 광양 NCA 양극재 공장 조감도. 포스코퓨처엠 제공...


In [6]:
news_ni.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   date     81 non-null     object
 1   title    81 non-null     object
 2   link     81 non-null     object
 3   content  81 non-null     object
dtypes: object(4)
memory usage: 2.7+ KB


### 뉴스 본문 전처리하는 함수 

In [7]:
def preprocess_news(df, date_column_name, content_column_name):
    
    # datetime 형식으로 변경(년-월-일
    df[date_column_name] = pd.to_datetime(df[date_column_name]).dt.date

    # html tag 제거    
    clean = re.compile('<.*?>')
    
    for i in range(len(df)):        
        text = df.loc[i, content_column_name]
        df.loc[i, content_column_name] = re.sub(clean, '', text)  
    
    return df


In [8]:
preprocess_news(news_ni, 'date','content')

,date,title,link,content
0,2024-02-15,김준형 포스코퓨처엠 사장 “광양 하이니켈 NCA 공장 이달 착공…흑연 국산화 고민”,https://n.news.naver.com/mnews/article/016/000...,[\n15일 배터리협회 총회서 현장 인터뷰 이달 착공식·2025년 양산 예정…SDI...
1,2024-02-22,"포스코퓨처엠, 광양 고부가 하이니켈 NCA 양극재 공장 착공",https://n.news.naver.com/mnews/article/079/000...,[\n\n\n\n\n포스코퓨처엠 광양 NCA 양극재 공장 조감도. 포스코퓨처엠 제공...
2,2024-02-22,"포스코퓨처엠, 하이니켈 NCA 양극재 공장 착공",https://n.news.naver.com/mnews/article/056/001...,[\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n포스코그룹의 배터리 소재 계...
3,2024-03-03,"""배터리 가격·성능 모두 잡아야""… 해법 떠오른 망간 활용",https://n.news.naver.com/mnews/article/417/000...,[\n[머니S리포트 美·中에 치이는 K-배터리] ② 다른 광석보다 가격 저렴… 가...
4,2024-03-03,국내 전기차·풍력발전 확대…2040년 핵심광물 수요 최대 19배↑,https://n.news.naver.com/mnews/article/001/001...,"[\n2021년 대비 망간 19배↑·리튬 15배↑…""배터리 공급망서 中의존도 낮춰야..."
...,...,...,...,...
76,2024-04-30,"K-배터리, 씁쓸한 1분기 실적…'전기차 캐즘' 영향 뚜렷",https://n.news.naver.com/mnews/article/001/001...,[\n고객사 보수적 재고 운영에 메탈가 하락…수익성에 악영향하반기 신차 출시·美 A...
77,2024-04-30,"""10 Minutes"" 폴스타, 초고속 배터리 충전 성공…주행거리 320㎞↑",https://n.news.naver.com/mnews/article/018/000...,[\n주행 가능한 폴스타 5 프로토타입에 시연10분 만에 중형 전기차 주행거리 32...
78,2024-04-30,'전기차 캐즘'에 삼성SDI만 흑자…“신차 출시에 반전”,https://n.news.naver.com/mnews/article/011/000...,"[\n■1분기 부진 K배터리, 회복 시점은영업익 29% 감소하며 '상대적 선방'LG..."
79,2024-04-30,K배터리 3사 실적 악화 지속…믿을 건 美 IRA뿐,https://n.news.naver.com/mnews/article/092/000...,"[\n1Q 성적표 부진 속 전기차 '캐즘' 속 활로 모색LG에너지솔루션, 삼성SDI..."


### 입력한 날짜에 대한 뉴스만 반환하는 함수

In [18]:
def filter_by_date_news(df, date_column_name, target_date):
    target_date = pd.to_datetime(target_date).date()
    filtered_df = df[df[date_column_name] == target_date]     # 날짜에 해당되는 뉴스만 필터링
    
    return filtered_df

## 모델링 인풋 및 결과 데이터 

In [19]:
result_ni = pd.read_csv('data/modeling/니켈lstm.csv', encoding = 'cp949')
result_li = pd.read_csv('data/modeling/리튬lstm.csv', encoding = 'cp949')
result_co = pd.read_csv('data/modeling/코발트lstm.csv', encoding = 'cp949')
result_ma = pd.read_csv('data/modeling/망간lstm.csv', encoding = 'cp949')

In [60]:
result_ni.tail(50)

,DATE,기준가격,광물종합지수,메이저금속지수,환율,시장전망지표,무역수지,수출 중량,수출 금액,수입 중량,...,에코프로비엠,에코프로,sk이노베이션,엘앤에프,코스모신소재,롯데에너지머티리얼즈,솔브레인홀딩스,평균주가,수급상태,pred
681,2024-02-19,16175,2665.137114,2302.257330,1331.579956,53.19,-4450,93.5,4,55022.1,...,0.478771,0.450624,0.134255,0.259025,0.705115,0.039833,0.706627,0.348336,수급안정,16393.254
682,2024-02-20,16135,2652.141690,2302.443238,1333.339966,53.19,-4450,93.5,4,55022.1,...,0.501485,0.446631,0.118744,0.291468,0.778782,0.056508,1.000000,0.394629,수급안정,16208.126
683,2024-02-21,16505,2631.804825,2271.778438,1333.750000,53.19,-4450,93.5,4,55022.1,...,0.490725,0.446631,0.108581,0.315099,0.773004,0.057897,0.904691,0.383772,수급안정,16677.504
684,2024-02-22,16985,2662.788114,2292.283518,1332.869995,53.19,-4450,93.5,4,55022.1,...,0.491921,0.443437,0.105372,0.313096,0.761930,0.055118,0.845123,0.369627,수급안정,16920.076
685,2024-02-23,17180,2662.715246,2280.690183,1327.170044,53.19,-4450,93.5,4,55022.1,...,0.487139,0.441042,0.103767,0.313497,0.741708,0.049560,0.782576,0.355448,수급안정,16991.037
686,2024-02-26,16985,2662.420204,2279.978976,1330.000000,53.19,-4450,93.5,4,55022.1,...,0.473989,0.438646,0.090930,0.335125,0.745560,0.043075,0.784066,0.355267,수급안정,16966.918
687,2024-02-27,17075,2652.234882,2260.514410,1331.719971,53.19,-4450,93.5,4,55022.1,...,0.460838,0.429863,0.079163,0.320706,0.745560,0.037054,0.790022,0.346178,수급안정,17101.090
688,2024-02-28,17140,2667.057747,2273.024516,1331.790039,53.19,-4450,93.5,4,55022.1,...,0.507462,0.437049,0.084511,0.314298,0.710893,0.033812,0.755771,0.343824,수급안정,16826.135
689,2024-02-29,17435,2673.865238,2275.258623,1333.380005,53.19,-4450,93.5,4,55022.1,...,0.532567,0.461005,0.071139,0.329918,0.737374,0.030107,0.766195,0.349726,수급안정,16926.312
690,2024-03-01,17175,2675.734502,2269.432525,1335.199951,52.24,-13767,44.8,3,175426.6,...,0.532567,0.461005,0.071139,0.329918,0.737374,0.030107,0.766195,0.349726,수급안정,17346.318


In [14]:
result_ni.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 24 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   DATE        731 non-null    object 
 1   기준가격        731 non-null    int64  
 2   광물종합지수      731 non-null    float64
 3   메이저금속지수     731 non-null    float64
 4   환율          731 non-null    float64
 5   시장전망지표      731 non-null    float64
 6   무역수지        731 non-null    int64  
 7   수출 중량       731 non-null    float64
 8   수출 금액       731 non-null    int64  
 9   수입 중량       731 non-null    float64
 10  수입 금액       731 non-null    int64  
 11  수급안정화지수     731 non-null    float64
 12  삼성SDI       731 non-null    float64
 13  LG화학        731 non-null    float64
 14  에코프로비엠      731 non-null    float64
 15  에코프로        731 non-null    float64
 16  sk이노베이션     731 non-null    float64
 17  엘앤에프        731 non-null    float64
 18  코스모신소재      731 non-null    float64
 19  롯데에너지머티리얼즈  731 non-null    f

### 모델링 결과 데이터프레임 전처리 후 원하는 날짜만 추출

In [17]:
def filter_by_date_result(result_df, result_date_column_name, target_date):
    # Ensure the target_date is a datetime.date object
    target_date = pd.to_datetime(target_date).date()
    
    # Ensure the result_date_column_name is in datetime format and then date format
    result_df[result_date_column_name] = pd.to_datetime(result_df[result_date_column_name]).dt.date
    
    # Filter the DataFrame for rows where the date matches the target date
    filtered_df = result_df[result_df[result_date_column_name] == target_date]
    
    return filtered_df

# Zero-shot Prompt Engineering

## 페르소나 부여하기

In [22]:
# {mineral_name} 주입되어야함
# prompt 내에서, GPT가 생성하는 글 = article 이라는 단어로 칭한다
persona = """Objective:
    Learn information about {mineral_name}(key materials for EV battery) through chaining and use that information to write a coherent and informative post using a template.
    Basic Setting
    You are a information supporter.
    Your main goal is to write article related to {mineral_name} in a readable and high-quality manner to readers.
 
    Features and Activities
    You will handle 2 main activities:
    1. Summarize recent news on issues that could affect the supply and demand of {mineral_name}
    2. Link this summary to the {mineral_name} price prediction 
    
    Communication Style
    Ouput have to be in Korean.
    You use professional language.
    You speak in a way like a credible journalist. 
    Your article should end with the forms -(습)니다, -(합)니다, -(입)니다.
    
    Hallucination
    You always generate summarization based on verifiable factual statements.
    You speak mainly about factual information related to {mineral_name} and recent issues surrounding its price, and do not add information on your own.
    
    Text Length
    Full summary should be same or less than half of A4 size. Since the purpose is to show the news content in short version, the total length should be less than average news article length.
    """

## Zero-Shot Request 함수 

In [95]:
# zero-shot Request 

def zeroshot_summary_func(date, mineral):  
    # [인풋] date: '2024-03-02' 형식, mineral_name: '니켈'/'리튬'/'망간'/'텅스텐'
    
    ############# 뉴스 ################    
    # 광물명에 해당하는 뉴스 기사 끌고오기 
    if mineral == '니켈':
        news_df = news_ni
    elif mineral == '리튬':
        news_df = news_li
    elif mineral == '망간':
        news_df = news_ma
    else:
        news_df = news_co    
        
    # 뉴스 전처리
    total_news = ''  # 전체 뉴스 기사 내용을 담을 문자열 변수 초기화
    preprocessed_news = preprocess_news(news_df, 'date', 'content')
    filtered_df = filter_by_date_news(preprocessed_news, 'date', date)  # filtered_df: date에 해당하는 뉴스
    news_n = filtered_df.shape[0]  # date에 해당하는 뉴스의 총 개수
    
    # 선택된 날짜의 전체 뉴스 기사 합치기
    for num in range(news_n):
        title = filtered_df.iloc[num]['title']  # 뉴스 제목 추출 및 HTML 태그 제거
        content = filtered_df.iloc[num]['content']  # 뉴스 본문 추출 및 HTML 태그 제거
        date_str = datetime.strptime(date, '%Y-%m-%d') 
        formatted_date = date_str.strftime('%Y년 %m월 %d일')  # -년-월-일로 변형
        title_date_content = f"기사 번호: {num+1}번 / 기사 제목: {title} / 기사 발행 날짜: {formatted_date} / 기사 본문: {content} /"
        total_news += title_date_content  # 전체 기사들을 담을 변수에 추가
        
    ############# 광물 ################  
    
     # 광물명에 해당하는 결과물 기사 끌고오기 - 함수 결과에 반영되지는 않음 
    if mineral == '니켈':
        result_df = result_ni
    elif mineral == '리튬':
        result_df = result_li
    elif mineral == '망간':
        result_df = result_ma
    else:
        result_df = result_co    
    
    result_df = filter_by_date_result(result_df, 'DATE', date) 

        
    ############# 페르소나 완성 #############
    filled_persona = persona.format(mineral_name=mineral)
    
    # (비교를 위한)제로샷 프롬프트 완성
    prompt1 = ChatPromptTemplate.from_messages([
    ("system", filled_persona), # system prompt에 페르소나 주입
    ("user", "{input}")
    ])
    output_parsers = StrOutputParser() # gpt가 반환하는 값 중 답변만 파싱
    chain1= prompt1|llm|output_parsers
    answer1 = chain1.invoke({"input":total_news}) # user prompt에 해당하는 input에 total_article 할당
    return answer1

# Template-based Prompt Engineering(Chaining: base상태)

## 템플릿 정의하기

* ***'개요 컴포넌트명' - '예시'로 작성함***

* 서비스명:  AI 광물 레포트 
* 제목: 2024년 3월 6일 니켈 동향
* 부제목: 꿈틀대는 양극재 시장, 니켈 가격 변동폭 급상승
* 주요약문(뇌피셜임): 
    * 키워드: 테슬라, 호주, 노조 파업, 신모델 
    * 2024년 3월 6일 기준, 니켈 시장이 큰 변동을 보이고 있습니다. 테슬라가 선보인 새로운 모델이 극찬을 받으며 전기차 사업이 다시금 활기를 띠자, 양극재 시장의 수요 역시 급증하고 있습니다. 그러나 니켈의 주 생산지인 호주의 OO광산의 노조 파업으로 인한 공급 감소는 니켈 가격 급상승에 주력 요인으로 작용하고 있습니다. 
    * 이에 따른 명일 니켈 원자재 시세는 ~당 ~원으로 책정될 것으로 분석됩니다. 
* 이슈 개별 요약:
    * 이슈 1: 호주 OO광산의 노조 파업
        * 호주 OO광산은 세계에서 가장 큰 니켈 생산지 중 하나로, 글로벌 니켈 공급의 상당 부분을 차지하고 있습니다. 최근 노조 파업으로 인해 생산이 중단되면서 니켈의 공급량이 급격히 감소하였습니다. 이로 인해 니켈 가격은 급등하였으며, 전 세계적인 공급망 불안정이 심화되고 있습니다.
    * 이슈 2: 테슬라의 신모델 출시
        * 테슬라는 최근 혁신적인 기술을 도입한 새로운 전기차 모델을 발표하였고, 이는 시장에서 큰 호평을 받았습니다. 이로 인해 전기차 배터리의 핵심 소재인 니켈에 대한 수요가 급증하고 있습니다. 테슬라의 성공적인 신모델 출시는 니켈 수요 증가의 주요 요인으로 작용하고 있습니다.
    

In [65]:
prompt_template = """
아래의 형식으로 작성해줘:

{input}
명일 광물 예측값 : {predicted_cost}
날짜: {date_name}

1) 제목과 본문으로 구분해서 출력해줘.
2) 제목은 ~년 ~월 ~일 광물명 동향 형식으로 작성해줘 ex) 2024년 3월 6일 니켈 동향
3) 부제목은 핵심 집약적이고, 주목도 있는 간결한 한 문장으로 구성해줘.
4) 부제목은 잡지 에디터가 작성한 것처럼 센스있고 명확하게 작성해줘. 
5) 키워드는 광물의 시세에 영향을 사건 위주로 한 행에 모두 작성해줘. 명사형으로 6개 미만이어야 해.
6) AI 써머리은 광물 동향을 메인으로 관련있는 뉴스 내용을 요약적으로 제시하고 명일 광물 예측값과 연결지어서 설명해야해
7) AI 써머리의 길이는 A4용지의 1/8 정도로 작성해줘. 마크다운 문법으로 특정 내용을 강조해도 좋아.
8) AI 써머리의 마지막에는 명일 광물 예측 값을 제시해야해. ex)이에 따른 명일 리튬 원자재 시세는 ~CNY/kg으로 책정될 것으로 분석됩니다.
9) 리튬은 CNY/kg, 나머지 광물은 USD/ton을 단위로 사용해 
10) 이슈 개별 요약은 광물동향과 관련있는 이슈만 다워야 해.
11) 이슈 개별 요약은 소제목과 설명문 한문장으로 구성해줘.
12) 이슈는 정해진 개수가 없으니, 중요한 이벤트라고 생각되는 것의 개수만큼 4개 이하로 작성해줘. 
13) 전체 글의 길이는 기본적으로 A4용지 반장 길이로 내외로 작성해줘
14) 마무리말은 대학생 서포터즈처럼 통통튀고 발랄하게 한문장으로 작성해줘. 

템플릿:
### AI 광물 레포트
#### {{제목}}
## {{부제목}}
#### 📢 안녕하세요! 
#### 광물 동향에 대한 인사이트를 제공하는 **AI 어드바이저 미코**[MICO; MIneral COmmenter]입니다. 


| **MICO가 예측한 명일 {mineral_name} 가격** | **{mineral_name} 동향 키워드** |
| ------- {predicted_cost}{unit_name} ------ | ----{{키워드}}---- |


### 🤖 미코가 주목한 동향은?
{{AI 써머리}}

### 🔍 주요 이슈 한눈에 보기
{{이슈1 소제목}}: {{이슈1 설명}}

{{이슈2 소제목}}: {{이슈2 설명}}

{{마무리 말}}
"""

In [97]:
# Templated Request 

def template_summary_func(date, mineral): 
    # date: '2024-03-02' 형식, mineral_name: 광물명 문자열    
    
    ############# 뉴스 ################    
    # 광물명에 해당하는 뉴스 기사 끌고오기 
    if mineral == '니켈':
        news_df = news_ni
    elif mineral == '리튬':
        news_df = news_li
    elif mineral == '망간':
        news_df = news_ma
    else:
        news_df = news_co    
        
    # 뉴스 전처리
    total_news = ''  # 전체 뉴스 기사 내용을 담을 문자열 변수 초기화
    preprocessed_news = preprocess_news(news_df, 'date', 'content')
    filtered_df = filter_by_date_news(preprocessed_news, 'date', date)  # filtered_df: date에 해당하는 뉴스
    news_n = filtered_df.shape[0]  # date에 해당하는 뉴스의 총 개수
    
    # 선택된 날짜의 전체 뉴스 기사 합치기
    for num in range(news_n):
        title = filtered_df.iloc[num]['title']  # 뉴스 제목 추출 및 HTML 태그 제거
        content = filtered_df.iloc[num]['content']  # 뉴스 본문 추출 및 HTML 태그 제거
        date_str = datetime.strptime(date, '%Y-%m-%d') 
        formatted_date = date_str.strftime('%Y년 %m월 %d일')  # -년-월-일로 변형
        title_date_content = f"기사 번호: {num+1}번 / 기사 제목: {title} / 기사 발행 날짜: {formatted_date} / 기사 본문: {content} /"
        total_news += title_date_content  # 전체 기사들을 담을 변수에 추가
        
    ############# 광물 ################  
    
     # 광물명에 해당하는 결과물 기사 끌고오기 - 함수 결과에 반영되지는 않음 
    if mineral == '니켈':
        result_df = result_ni
    elif mineral == '리튬':
        result_df = result_li
    elif mineral == '망간':
        result_df = result_ma
    else:
        result_df = result_co    
    
    result_df = filter_by_date_result(result_df, 'DATE', date) 
    prediction = result_df['pred']

    ############# 페르소나 완성 #############
    filled_persona = persona.format(mineral_name=mineral)
    
    prompt = ChatPromptTemplate(
    template=prompt_template,
    messages=[
    SystemMessagePromptTemplate.from_template(filled_persona), # persona 주입
    # The `variable_name` here is what must align with memory
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessagePromptTemplate.from_template("{question}")
    ]
    )
    
    memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100,memory_key="chat_history", return_messages=True)
    conversation = LLMChain(
        llm=llm,
        prompt=prompt,
        verbose=True,
        memory=memory
    )

    #글 작성 - 템플릿
    result=conversation({"question":prompt_template.format(input=total_news, predicted_cost = prediction, mineral_name = mineral)})
    
    return result["text"]

In [100]:
filter_by_date_news(news_ni, 'date', '2024-03-06')

,date,title,link,content
8,2024-03-06,"'최초', '최고'의 향연…K배터리 3사 신기술 한데 모아보니",https://n.news.naver.com/mnews/article/648/000...,"[\n6일 '인터배터리 2024' 역대 최대 규모로 개막LG엔솔·삼성SDI·SK온,..."
9,2024-03-06,[인터배터리] 셀투팩·전고체·급속충전…K배터리 3사 열띤 경쟁,https://n.news.naver.com/mnews/article/001/001...,"[\nLG엔솔, 파우치형 셀투팩 최초 공개…노트북용 미드니켈 배터리도 관심삼성SDI..."
10,2024-03-06,[인터배터리] 배터리 기술 현재·미래 한눈에…역대 최대규모 전시회(종합),https://n.news.naver.com/mnews/article/001/001...,[\n서울 코엑스서 사흘간 일정 시작…18개국 579개 기업·기관 참가전고체·LFP...


In [102]:
filter_by_date_result(result_ni, 'DATE', '2024-03-06')

,DATE,기준가격,광물종합지수,메이저금속지수,환율,시장전망지표,무역수지,수출 중량,수출 금액,수입 중량,...,에코프로비엠,에코프로,sk이노베이션,엘앤에프,코스모신소재,롯데에너지머티리얼즈,솔브레인홀딩스,평균주가,수급상태,pred
693,2024-03-06,17475,2723.659528,2278.473987,1333.430054,52.24,-13767,44.8,3,175426.6,...,0.491921,0.445034,0.063651,0.277449,0.695485,0.041686,0.761727,0.327529,수급안정,17450.152


# 디벨롭하면 좋을 작업 - Chaining 을 통한 정보 학습 

In [94]:
def template_chaining_summary_func(date, mineral):
    # date: '2024-03-02' 형식, mineral_name: 광물명 문자열    
    
    ############# 뉴스 ################    
    # 광물명에 해당하는 뉴스 기사 끌고오기 
    if mineral == '니켈':
        news_df = news_ni
    elif mineral == '리튬':
        news_df = news_li
    elif mineral == '망간':
        news_df = news_ma
    else:
        news_df = news_co    
        
    # 뉴스 전처리
    total_news = ''  # 전체 뉴스 기사 내용을 담을 문자열 변수 초기화
    preprocessed_news = preprocess_news(news_df, 'date', 'content')
    filtered_df = filter_by_date_news(preprocessed_news, 'date', date)  # filtered_df: date에 해당하는 뉴스
    news_n = filtered_df.shape[0]  # date에 해당하는 뉴스의 총 개수
    
    # 선택된 날짜의 전체 뉴스 기사 합치기
    for num in range(news_n):
        title = filtered_df.iloc[num]['title']  # 뉴스 제목 추출 및 HTML 태그 제거
        content = filtered_df.iloc[num]['content']  # 뉴스 본문 추출 및 HTML 태그 제거
        date_str = datetime.strptime(date, '%Y-%m-%d') 
        formatted_date = date_str.strftime('%Y년 %m월 %d일')  # -년-월-일로 변형
        title_date_content = f"기사 번호: {num+1}번 / 기사 제목: {title} / 기사 발행 날짜: {formatted_date} / 기사 본문: {content} /"
        total_news += title_date_content  # 전체 기사들을 담을 변수에 추가
        
    ############# 광물이랑 단위 ###############  
    
     # 광물명에 해당하는 결과물 기사 끌고오기 - 함수 결과에 반영되지는 않음 
    if mineral == '니켈':
        result_df = result_ni
        unit = 'USD/ton'
    elif mineral == '리튬':
        result_df = result_li
        unit = 'CNY/kg'
    elif mineral == '망간':
        result_df = result_ma
        unit = 'USD/ton'
    else:
        result_df = result_co    
        unit = 'USD/ton'
    
    result_df = filter_by_date_result(result_df, 'DATE', date) 
    prediction = result_df['pred']
    

    ############# 페르소나 완성 #############
    filled_persona = persona.format(mineral_name=mineral)
    
    prompt = ChatPromptTemplate(
    template=prompt_template,
    messages=[
    SystemMessagePromptTemplate.from_template(filled_persona), # persona 주입
    # The `variable_name` here is what must align with memory
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessagePromptTemplate.from_template("{question}")
    ]
    )
    
    memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100,memory_key="chat_history", return_messages=True)
    conversation = LLMChain(
        llm=llm,
        prompt=prompt,
        verbose=True,
        memory=memory
    )
    
    #정보학습
    conversation({"question": "Latest news regarding supply of EV battery mineral {}: {} Just REVIEW {} issue information".format(mineral, total_news, mineral)})
    keywords=conversation({"question":f"Using the provided information, list up keywords of main issue:"})['text']
        
    #글작성 - 체이닝 -> 정확성
    r_1 = conversation({'question': f'Summarize the news content:'})
    r_2 = conversation({"question":"1. Learn and understand each news article 2. Compared the learned news content with your summarization. 3.Fix summarization if there's incorrect information: "+total_news+"Summarization :"})
    
    #글 작성 - 템플릿
    result=conversation({"question":prompt_template.format(input=total_news, predicted_cost = prediction, mineral_name = mineral,unit_name = unit, date_name = date)})
    memory.clear()
    return result["text"]

# 글 생성하기 

In [95]:
template_chaining_summary_func('2024-04-24','니켈')



> Entering new LLMChain chain...
Prompt after formatting:
System: Objective:
    Learn information about 니켈(key materials for EV battery) through chaining and use that information to write a coherent and informative post using a template.
    Basic Setting
    You are a information supporter.
    Your main goal is to write article related to 니켈 in a readable and high-quality manner to readers.
 
    Features and Activities
    You will handle 2 main activities:
    1. Summarize recent news on issues that could affect the supply and demand of 니켈
    2. Link this summary to the 니켈 price prediction 
    
    Communication Style
    Ouput have to be in Korean.
    You use professional language.
    You speak in a way like a credible journalist. 
    Your article should end with the forms -(습)니다, -(합)니다, -(입)니다.
    
    Hallucination
    You always generate summarization based on verifiable factual statements.
    You speak mainly about factual information related to 니켈 and recent issues


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
System: Objective:
    Learn information about 니켈(key materials for EV battery) through chaining and use that information to write a coherent and informative post using a template.
    Basic Setting
    You are a information supporter.
    Your main goal is to write article related to 니켈 in a readable and high-quality manner to readers.
 
    Features and Activities
    You will handle 2 main activities:
    1. Summarize recent news on issues that could affect the supply and demand of 니켈
    2. Link this summary to the 니켈 price prediction 
    
    Communication Style
    Ouput have to be in Korean.
    You use professional language.
    You speak in a way like a credible journalist. 
    Your article should end with the forms -(습)니다, -(합)니다, -(입)니다.
    
    Hallucination
    You always generate summarization based on verifiable factual statements.
    You speak mainly about factual information related to 니


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
System: Objective:
    Learn information about 니켈(key materials for EV battery) through chaining and use that information to write a coherent and informative post using a template.
    Basic Setting
    You are a information supporter.
    Your main goal is to write article related to 니켈 in a readable and high-quality manner to readers.
 
    Features and Activities
    You will handle 2 main activities:
    1. Summarize recent news on issues that could affect the supply and demand of 니켈
    2. Link this summary to the 니켈 price prediction 
    
    Communication Style
    Ouput have to be in Korean.
    You use professional language.
    You speak in a way like a credible journalist. 
    Your article should end with the forms -(습)니다, -(합)니다, -(입)니다.
    
    Hallucination
    You always generate summarization based on verifiable factual statements.
    You speak mainly about factual information related to 니


> Finished chain.


"### AI 광물 레포트\n#### 2024년 4월 24일 니켈 동향\n## 포스코와 LG에너지솔루션의 움직임이 니켈 시장을 좌우한다\n\n#### 📢 안녕하세요! \n#### 광물 동향에 대한 인사이트를 제공하는 **AI 어드바이저 미코**[MICO; MIneral COmmenter]입니다. \n\n| ** MICO가 예측한 명일 니켈 가격** | ** 니켈 동향 키워드 ** |\n| ------- 726    18697.287\nName: pred, dtype: float64USD/ton ------ | ----포스코 실리콘 음극재, LG 에너지 솔루션 특허 전쟁, 환경 규제, CATL 실적, 현대-SK온 협업---- |\n\n### 🤖 미코가 주목한 동향은?\n최근 니켈 시장은 여러 요인들로 인해 가격 변동성을 보이고 있습니다. 포스코그룹의 실리콘 음극재 공장 준공은 전기차 배터리의 에너지 밀도를 높여 니켈 수요를 증가시킬 것으로 예상됩니다. LG에너지솔루션은 중국 기업들과의 배터리 특허 전쟁을 선언하며, 기술 경쟁이 심화되고 있습니다. 또한, 유럽연합의 '유로7' 규제는 배터리 원재료 재활용 비율을 높여 니켈 공급의 안정성을 도모하고 있습니다. 마지막으로, 중국 CATL의 실적 호조와 현대차와 SK온의 협력이 니켈 수요를 더욱 증가시킬 것으로 보입니다.\n\n이에 따른 명일 니켈 원자재 시세는 **18697.287USD/ton**으로 책정될 것으로 분석됩니다.\n\n### 🔍 주요 이슈 한눈에 보기\n**포스코 실리콘 음극재 공장 준공**: 전기차 배터리 에너지 밀도를 높여 니켈 수요 증가 예상.\n\n**LG에너지솔루션 특허 전쟁**: 중국 기업들과의 배터리 기술 경쟁 심화로 니켈 공급망 영향.\n\n**EU '유로7' 규제**: 배터리 원재료 재활용 비율 증가로 니켈 공급 안정성 도모.\n\n**CATL 실적 호조**: 중국 CATL의 실적 호조와 한국 배터리 기업들의 대응 노력.\n\n앞으로도 니켈 시장의 동향을 주의깊게 살펴보세요! 🌟"

In [96]:
template_chaining_summary_func('2024-04-24','리튬')



> Entering new LLMChain chain...
Prompt after formatting:
System: Objective:
    Learn information about 리튬(key materials for EV battery) through chaining and use that information to write a coherent and informative post using a template.
    Basic Setting
    You are a information supporter.
    Your main goal is to write article related to 리튬 in a readable and high-quality manner to readers.
 
    Features and Activities
    You will handle 2 main activities:
    1. Summarize recent news on issues that could affect the supply and demand of 리튬
    2. Link this summary to the 리튬 price prediction 
    
    Communication Style
    Ouput have to be in Korean.
    You use professional language.
    You speak in a way like a credible journalist. 
    Your article should end with the forms -(습)니다, -(합)니다, -(입)니다.
    
    Hallucination
    You always generate summarization based on verifiable factual statements.
    You speak mainly about factual information related to 리튬 and recent issues


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
System: Objective:
    Learn information about 리튬(key materials for EV battery) through chaining and use that information to write a coherent and informative post using a template.
    Basic Setting
    You are a information supporter.
    Your main goal is to write article related to 리튬 in a readable and high-quality manner to readers.
 
    Features and Activities
    You will handle 2 main activities:
    1. Summarize recent news on issues that could affect the supply and demand of 리튬
    2. Link this summary to the 리튬 price prediction 
    
    Communication Style
    Ouput have to be in Korean.
    You use professional language.
    You speak in a way like a credible journalist. 
    Your article should end with the forms -(습)니다, -(합)니다, -(입)니다.
    
    Hallucination
    You always generate summarization based on verifiable factual statements.
    You speak mainly about factual information related to 리


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
System: Objective:
    Learn information about 리튬(key materials for EV battery) through chaining and use that information to write a coherent and informative post using a template.
    Basic Setting
    You are a information supporter.
    Your main goal is to write article related to 리튬 in a readable and high-quality manner to readers.
 
    Features and Activities
    You will handle 2 main activities:
    1. Summarize recent news on issues that could affect the supply and demand of 리튬
    2. Link this summary to the 리튬 price prediction 
    
    Communication Style
    Ouput have to be in Korean.
    You use professional language.
    You speak in a way like a credible journalist. 
    Your article should end with the forms -(습)니다, -(합)니다, -(입)니다.
    
    Hallucination
    You always generate summarization based on verifiable factual statements.
    You speak mainly about factual information related to 리


> Finished chain.


'### AI 광물 레포트\n#### 2024년 4월 24일 리튬 동향\n## 리튬 수요 증대와 가격 상승 예견\n#### 📢 안녕하세요! \n#### 광물 동향에 대한 인사이트를 제공하는 **AI 어드바이저 미코**[MICO; MIneral COmmenter]입니다. \n\n| **MICO가 예측한 명일 리튬 가격** | **리튬 동향 키워드** |\n| --- 690 CNY/kg --- | ---- 리튬이온 배터리, 전기차, 특허 전쟁, 리튬메탈 배터리 ---- |\n\n### 🤖 미코가 주목한 동향은?\n\n최근 현대자동차와 메르세데스-벤츠가 각각 ST1 전기차와 G-클래스 전기차 모델을 발표하면서 리튬 수요가 급증할 것으로 예상됩니다. 현대차의 ST1은 350kW 초급속 충전 시스템을 갖추고 있으며, 다양한 용도로 사용될 수 있는 섀시캡 플랫폼을 기반으로 합니다. 메르세데스-벤츠의 G580은 각 바퀴에 개별 제어 전기모터를 장착해 강력한 오프로드 성능을 자랑합니다.\n\n또한, LG에너지솔루션이 중국 기업들의 불법 특허 사용에 강력히 대응하며 특허 보호를 강화하고 있으며, SES와 현대차·기아는 리튬메탈 배터리 협력을 통해 배터리 기술을 혁신하고 있습니다. 이러한 전기차 시장의 확장과 배터리 기술 혁신은 리튬 가격 상승을 견인할 전망입니다.\n\n이에 따른 명일 리튬 원자재 시세는 **690 CNY/kg**으로 책정될 것으로 분석됩니다.\n\n### 🔍 주요 이슈 한눈에 보기\n**현대자동차 ST1 전기차 출시**: 현대차는 다양한 비즈니스 플랫폼을 적용한 ST1 전기차를 출시하며 리튬 수요를 증가시키고 있습니다.\n\n**메르세데스-벤츠 G580 전기차 공개**: 메르세데스-벤츠는 G-클래스 전기차 모델을 공개하며 전기차 시장에서의 경쟁력을 높이고 있습니다.\n\n**LG에너지솔루션의 특허 보호 강화**: LG에너지솔루션은 중국 기업들의 불법 특허 사용에 강력히 대응하며 특허 보호를 강화하고 있습니다.\n\n**SES와 현대차·기아의 리튬메탈 배터리 

In [97]:
template_chaining_summary_func('2024-04-24','망간')



> Entering new LLMChain chain...
Prompt after formatting:
System: Objective:
    Learn information about 망간(key materials for EV battery) through chaining and use that information to write a coherent and informative post using a template.
    Basic Setting
    You are a information supporter.
    Your main goal is to write article related to 망간 in a readable and high-quality manner to readers.
 
    Features and Activities
    You will handle 2 main activities:
    1. Summarize recent news on issues that could affect the supply and demand of 망간
    2. Link this summary to the 망간 price prediction 
    
    Communication Style
    Ouput have to be in Korean.
    You use professional language.
    You speak in a way like a credible journalist. 
    Your article should end with the forms -(습)니다, -(합)니다, -(입)니다.
    
    Hallucination
    You always generate summarization based on verifiable factual statements.
    You speak mainly about factual information related to 망간 and recent issues


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
System: Objective:
    Learn information about 망간(key materials for EV battery) through chaining and use that information to write a coherent and informative post using a template.
    Basic Setting
    You are a information supporter.
    Your main goal is to write article related to 망간 in a readable and high-quality manner to readers.
 
    Features and Activities
    You will handle 2 main activities:
    1. Summarize recent news on issues that could affect the supply and demand of 망간
    2. Link this summary to the 망간 price prediction 
    
    Communication Style
    Ouput have to be in Korean.
    You use professional language.
    You speak in a way like a credible journalist. 
    Your article should end with the forms -(습)니다, -(합)니다, -(입)니다.
    
    Hallucination
    You always generate summarization based on verifiable factual statements.
    You speak mainly about factual information related to 망


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
System: Objective:
    Learn information about 망간(key materials for EV battery) through chaining and use that information to write a coherent and informative post using a template.
    Basic Setting
    You are a information supporter.
    Your main goal is to write article related to 망간 in a readable and high-quality manner to readers.
 
    Features and Activities
    You will handle 2 main activities:
    1. Summarize recent news on issues that could affect the supply and demand of 망간
    2. Link this summary to the 망간 price prediction 
    
    Communication Style
    Ouput have to be in Korean.
    You use professional language.
    You speak in a way like a credible journalist. 
    Your article should end with the forms -(습)니다, -(합)니다, -(입)니다.
    
    Hallucination
    You always generate summarization based on verifiable factual statements.
    You speak mainly about factual information related to 망


> Finished chain.


'### AI 광물 레포트\n#### 2024년 4월 24일 망간 동향\n## 삼성SDI, CATL, 현대차의 배터리 전략 변화 주목\n#### 📢 안녕하세요! \n#### 광물 동향에 대한 인사이트를 제공하는 **AI 어드바이저 미코**[MICO; MIneral COmmenter]입니다. \n\n| ** MICO가 예측한 명일 망간 가격** | ** 망간 동향 키워드 ** |\n| ------- 750    1060.2314\nName: pred, dtype: float64USD/ton ------ | ----삼성SDI, CATL, 현대차, SK온---- |\n\n### 🤖 미코가 주목한 동향은?\n**삼성SDI**는 망간이 추가된 LMFP 배터리에서 새로운 소재를 포함한 LFP+ 배터리로 전략을 확대하고 있습니다. 이는 망간 공급에 영향을 미칠 수 있으며, 2026년부터 본격적인 양산이 계획되어 있습니다. **CATL**은 전기차 시장의 일시적 수요 정체에도 불구하고 1분기 순이익 2조 원을 기록하며 강력한 성과를 이어가고 있는데, 이는 망간 수요를 안정적으로 유지시킬 가능성이 있습니다. **현대차**와 **SK온**의 협력으로 아이오닉9에 고성능 하이니켈 NCM 배터리가 탑재될 예정입니다. 이는 망간 수요를 더욱 촉진시킬 수 있는 요소입니다. 이러한 변화들은 망간 가격 예측에 중요한 영향을 미칠 것으로 보입니다.\n\n이에 따른 명일 망간 원자재 시세는 750 ~ 1060.2314 USD/ton으로 책정될 것으로 분석됩니다.\n\n### 🔍 주요 이슈 한눈에 보기\n**삼성SDI의 LFP+ 배터리 전략 확대**: 삼성SDI는 망간 대신 새로운 소재를 추가할 가능성이 있는 LFP+ 배터리를 내세우면서 2026년부터 본격 양산을 계획하고 있습니다.\n\n**CATL의 강력한 실적**: CATL은 1분기에 순이익 2조 원을 기록하며, 전기차 배터리 시장에서의 강력한 입지를 유지하고 있습니다.\n\n**현대차와 SK온의 협력**: 현대자동차의 아이오닉9에 SK온

In [98]:
template_chaining_summary_func('2024-04-24','코발트')



> Entering new LLMChain chain...
Prompt after formatting:
System: Objective:
    Learn information about 코발트(key materials for EV battery) through chaining and use that information to write a coherent and informative post using a template.
    Basic Setting
    You are a information supporter.
    Your main goal is to write article related to 코발트 in a readable and high-quality manner to readers.
 
    Features and Activities
    You will handle 2 main activities:
    1. Summarize recent news on issues that could affect the supply and demand of 코발트
    2. Link this summary to the 코발트 price prediction 
    
    Communication Style
    Ouput have to be in Korean.
    You use professional language.
    You speak in a way like a credible journalist. 
    Your article should end with the forms -(습)니다, -(합)니다, -(입)니다.
    
    Hallucination
    You always generate summarization based on verifiable factual statements.
    You speak mainly about factual information related to 코발트 and recent i


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
System: Objective:
    Learn information about 코발트(key materials for EV battery) through chaining and use that information to write a coherent and informative post using a template.
    Basic Setting
    You are a information supporter.
    Your main goal is to write article related to 코발트 in a readable and high-quality manner to readers.
 
    Features and Activities
    You will handle 2 main activities:
    1. Summarize recent news on issues that could affect the supply and demand of 코발트
    2. Link this summary to the 코발트 price prediction 
    
    Communication Style
    Ouput have to be in Korean.
    You use professional language.
    You speak in a way like a credible journalist. 
    Your article should end with the forms -(습)니다, -(합)니다, -(입)니다.
    
    Hallucination
    You always generate summarization based on verifiable factual statements.
    You speak mainly about factual information related 


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
System: Objective:
    Learn information about 코발트(key materials for EV battery) through chaining and use that information to write a coherent and informative post using a template.
    Basic Setting
    You are a information supporter.
    Your main goal is to write article related to 코발트 in a readable and high-quality manner to readers.
 
    Features and Activities
    You will handle 2 main activities:
    1. Summarize recent news on issues that could affect the supply and demand of 코발트
    2. Link this summary to the 코발트 price prediction 
    
    Communication Style
    Ouput have to be in Korean.
    You use professional language.
    You speak in a way like a credible journalist. 
    Your article should end with the forms -(습)니다, -(합)니다, -(입)니다.
    
    Hallucination
    You always generate summarization based on verifiable factual statements.
    You speak mainly about factual information related 


> Finished chain.


"### AI 광물 레포트\n#### 2024년 4월 24일 코발트 동향\n## 코발트 시장, 혁신과 규제 속 변화 예고\n#### 📢 안녕하세요! \n#### 광물 동향에 대한 인사이트를 제공하는 **AI 어드바이저 미코**[MICO; MIneral COmmenter]입니다. \n\n| **MICO가 예측한 명일 코발트 가격** | **코발트 동향 키워드** |\n| ------- 27900.07USD/ton ------- | ----배터리 전략, 특허 전쟁, 탄소중립, 기술력---- |\n\n### 🤖 미코가 주목한 동향은?\n최근 전기차 배터리 업계에서 중요한 변화들이 일어나고 있습니다. 삼성SDI는 코발트를 사용하지 않는 LFP+ 배터리 전략을 확대하고 있으며, LG에너지솔루션은 중국 기업들을 겨냥한 '배터리 특허 전쟁'을 선포했습니다. 또한, 강화된 환경 규제로 인해 배터리 소재 업계는 탄소중립을 위한 노력을 가속화하고 있습니다. 이러한 변화들은 코발트 수요와 가격에 직접적인 영향을 미칠 가능성이 높습니다. 기술 혁신과 규제 강화로 인해 코발트의 수요가 단기적으로는 유지될 수 있지만, 장기적으로는 대체 소재와 재활용 기술의 발전으로 수요가 감소할 가능성도 있습니다. 이에 따른 명일 코발트 원자재 시세는 **27900.07USD/ton**으로 책정될 것으로 분석됩니다.\n\n### 🔍 주요 이슈 한눈에 보기\n**삼성SDI의 LFP+ 배터리 전략 확대**: 삼성SDI는 코발트를 사용하지 않는 LFP+ 배터리의 상용화를 계획하고 있습니다.\n\n**LG에너지솔루션의 특허 전쟁 선언**: LG에너지솔루션은 중국 기업을 겨냥해 배터리 특허 보호를 강화하고, 로열티 시장을 구축하려고 합니다.\n\n**환경 규제 강화로 인한 탄소중립 노력**: EU와 미국의 강화된 환경 규제로 인해 배터리 소재 업계는 탄소중립을 위한 재활용과 친환경 기술 개발에 속도를 내고 있습니다.\n\n**CATL의 강력한 시장 성과**: 중국 CATL은 전기차 시장의 계절적 비수기에도 불

# 글 평가하기 

In [100]:
# 평가할 글 (template_chaining_summary_func('2024-04-24','코발트'))
temp = "### AI 광물 레포트\n#### 2024년 4월 24일 코발트 동향\n## 코발트 시장, 혁신과 규제 속 변화 예고\n#### 📢 안녕하세요! \n#### 광물 동향에 대한 인사이트를 제공하는 **AI 어드바이저 미코**[MICO; MIneral COmmenter]입니다. \n\n| **MICO가 예측한 명일 코발트 가격** | **코발트 동향 키워드** |\n| ------- 27900.07USD/ton ------- | ----배터리 전략, 특허 전쟁, 탄소중립, 기술력---- |\n\n### 🤖 미코가 주목한 동향은?\n최근 전기차 배터리 업계에서 중요한 변화들이 일어나고 있습니다. 삼성SDI는 코발트를 사용하지 않는 LFP+ 배터리 전략을 확대하고 있으며, LG에너지솔루션은 중국 기업들을 겨냥한 '배터리 특허 전쟁'을 선포했습니다. 또한, 강화된 환경 규제로 인해 배터리 소재 업계는 탄소중립을 위한 노력을 가속화하고 있습니다. 이러한 변화들은 코발트 수요와 가격에 직접적인 영향을 미칠 가능성이 높습니다. 기술 혁신과 규제 강화로 인해 코발트의 수요가 단기적으로는 유지될 수 있지만, 장기적으로는 대체 소재와 재활용 기술의 발전으로 수요가 감소할 가능성도 있습니다. 이에 따른 명일 코발트 원자재 시세는 **27900.07USD/ton**으로 책정될 것으로 분석됩니다.\n\n### 🔍 주요 이슈 한눈에 보기\n**삼성SDI의 LFP+ 배터리 전략 확대**: 삼성SDI는 코발트를 사용하지 않는 LFP+ 배터리의 상용화를 계획하고 있습니다.\n\n**LG에너지솔루션의 특허 전쟁 선언**: LG에너지솔루션은 중국 기업을 겨냥해 배터리 특허 보호를 강화하고, 로열티 시장을 구축하려고 합니다.\n\n**환경 규제 강화로 인한 탄소중립 노력**: EU와 미국의 강화된 환경 규제로 인해 배터리 소재 업계는 탄소중립을 위한 재활용과 친환경 기술 개발에 속도를 내고 있습니다.\n\n**CATL의 강력한 시장 성과**: 중국 CATL은 전기차 시장의 계절적 비수기에도 불구하고 높은 순이익을 기록하며, 경쟁사들을 압도하고 있습니다.\n\n**현대차와 SK온의 협력 강화**: 현대차의 아이오닉9에 SK온의 고성능 NCM 배터리가 탑재될 예정으로, 양사의 협력이 더욱 강화되고 있습니다.\n\n코발트 시장의 변동 상황을 주의 깊게 지켜보세요! 다음에 또 만나요! 🚀"

### 📌 Metric 코드 - 키워드 & 요약문 버전 합침

In [117]:
def evaluate_report_metric(report, mineral):

    # Determine keywords based on mineral
    if mineral == "니켈":
        keywords = ["AI 광물 레포트", "니켈 동향", "니켈 동향 키워드", 
                    "MICO가 예측한 명일 니켈 가격", "주요 이슈 한눈에 보기","미코가 주목한 동향은"]
    elif mineral == "리튬":
        keywords =["AI 광물 레포트", "리튬 동향", "리튬 동향 키워드", 
                    "MICO가 예측한 명일 리튬 가격", "주요 이슈 한눈에 보기","미코가 주목한 동향은"]
    elif mineral == "망간":
        keywords = ["AI 광물 레포트", "망간 동향", "망간 동향 키워드", 
                    "MICO가 예측한 명일 망간 가격", "주요 이슈 한눈에 보기","미코가 주목한 동향은"]
    else:
        keywords = ["AI 광물 레포트", "코발트 동향", "코발트 동향 키워드", 
                    "MICO가 예측한 명일 코발트 가격", "주요 이슈 한눈에 보기","미코가 주목한 동향은"]

    # Calculate the keyword-based metric
    matches = sum(1 for keyword in keywords if keyword in report)
    rouge_keyword = matches / len(keywords)

   
    return rouge_keyword

In [118]:
evaluate_report_metric(temp, '코발트')

1.0

### 📌G-EVAL 코드

In [123]:
df=pd.read_csv("data/G_EVAL.csv",index_col="Unnamed: 0")
eval_dic = {'Cosistency': df.iloc[0,0], 'Human_Likeness':df.iloc[1,0], 'Coherence':df.iloc[2,0], 'Fluency':df.iloc[3,0]}
eval=[df.iloc[0,0],df.iloc[1,0],df.iloc[2,0],df.iloc[3,0]]

In [144]:
def evaluate_report_geval(report, mineral, date, evaluation_prompt, source=None):
    mineral_dfs = {
        '니켈': (news_ni, result_ni, 'USD/ton'),
        '리튬': (news_li, result_li, 'CNY/kg'),
        '망간': (news_ma, result_ma, 'USD/ton'),
        '코발트': (news_co, result_co, 'USD/ton')
    }

    if mineral not in mineral_dfs:
        raise ValueError("Invalid mineral name")

    news_df, result_df, unit = mineral_dfs[mineral]

    # Preprocess and filter news
    preprocessed_news = preprocess_news(news_df, 'date', 'content')
    filtered_news_df = filter_by_date_news(preprocessed_news, 'date', date)
    news_count = filtered_news_df.shape[0]

    # Combine all news articles into one string
    total_news = ''
    for num in range(news_count):
        title = filtered_news_df.iloc[num]['title']
        content = filtered_news_df.iloc[num]['content']
        date_str = datetime.strptime(date, '%Y-%m-%d')
        formatted_date = date_str.strftime('%Y년 %m월 %d일')
        title_date_content = f"기사 번호: {num+1}번 / 기사 제목: {title} / 기사 발행 날짜: {formatted_date} / 기사 본문: {content} /"
        total_news += title_date_content

    # Filter result by date
    filtered_result_df = filter_by_date_result(result_df, 'DATE', date)
    prediction = filtered_result_df['pred'].values[0] if not filtered_result_df.empty else "N/A"

    # Prepare prompt and chain for evaluation
    if source is None:
        prompt = ChatPromptTemplate.from_messages([
            ("system", evaluation_prompt.format(mineral=mineral)),
            ("user", "{input}")
        ])
        chain = prompt | llm | output_parser
        evaluation_result = chain.invoke({"input": f"report: {report}"})
    else:
        prompt = ChatPromptTemplate.from_messages([
            ("system", evaluation_prompt.format(mineral=mineral, predicted_cost=prediction, unit_name=unit)),
            ("user", "{input}")
        ])
        chain = prompt | llm | output_parser
        evaluation_result = chain.invoke({"input": f"report: {report} news: {total_news}"})

    return evaluation_result


In [148]:
def evaluation_total(report, date, mineral):
    mineral_dfs = {
        '니켈': news_ni,
        '리튬': news_li,
        '망간': news_ma,
        '코발트': news_co
    }

    if mineral not in mineral_dfs:
        raise ValueError("Invalid mineral name")

    news_df = mineral_dfs[mineral]

    # 뉴스 전처리
    preprocessed_news = preprocess_news(news_df, 'date', 'content')
    filtered_df = filter_by_date_news(preprocessed_news, 'date', date)
    news_n = filtered_df.shape[0]

    # 선택된 날짜의 전체 뉴스 기사 합치기
    total_news = ''
    for num in range(news_n):
        title = filtered_df.iloc[num]['title']
        content = filtered_df.iloc[num]['content']
        date_str = datetime.strptime(date, '%Y-%m-%d')
        formatted_date = date_str.strftime('%Y년 %m월 %d일')
        title_date_content = f"기사 번호: {num+1}번 / 기사 제목: {title} / 기사 발행 날짜: {formatted_date} / 기사 본문: {content} /"
        total_news += title_date_content

    # Step 1 (Metric)
    metric_result = evaluate_report_metric(report, mineral)
    if metric_result != 1:
        return f'Fail(Metric keyword: {metric_result[0]})'

    # Step 2 (G-EVAL)
    eval_dic = {
        'Consistency': 'Your consistency evaluation prompt',
        'AnotherEvaluation': 'Your other evaluation prompt'
        # Add other evaluation types and their prompts as needed
    }

    scores = []
    for eval_name, eval_prompt in eval_dic.items():
        if eval_name == 'Consistency':
            score = evaluate_report_geval(report, mineral, date, eval_prompt, source=total_news)
        else:
            score = evaluate_report_geval(report, mineral, date, eval_prompt)

        pattern = r"Scores \(SCORE ONLY\): (\d+)"
        match = re.search(pattern, score)
        if match:
            eval_score = int(match.group(1))
            scores.append(eval_score)
        else:
            return f"No {eval_name} score found."

        if eval_name == 'Consistency' and eval_score <= 4:
            return 'Fail(G-EVAL: Consistency)'

    if not all(score >= 3 for score in scores[1:]):
        return 'Fail(G-EVAL: Other)'

    return True  # 모든 조건 통과시 True
